In [33]:
import numpy as np
import pandas as pd 

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/clean-data-australia-rainfall/rainfall_data_australia.csv', index_col=None)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isna().any()

In [ ]:
df = df.dropna()

In [ ]:
df.isna().any()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['RainTomorrow'].unique()

In [ ]:
string_var = {'RainTomorrow': {1:'Yes', 2:'No', 3:'Yes_No'}}

df = df.replace(string_var)

In [ ]:
x_cols = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday']

In [ ]:
x = df[x_cols]

y = df[['RainTomorrow']]

In [ ]:
x

In [ ]:
scaler = MinMaxScaler().fit(x)

x = scaler.transform(x)

In [ ]:
x

In [ ]:
y

In [ ]:
#y_ = y.toarray()

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(y)

y = encoder.transform(y)

In [ ]:
y

In [ ]:
x_ = pd.DataFrame(x)

In [ ]:
X = torch.tensor(x_.values, dtype=torch.float32)

In [ ]:
y_ = pd.DataFrame(y)

In [ ]:
y_

In [ ]:
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
class Multiclass(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(17, 34)
        self.act = nn.ReLU()
        self.output = nn.Linear(34, 3)
        
    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        
        return x

In [ ]:
model = Multiclass()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
n_epochs = 5
batch_size = 5
batches_per_epoch = len(x_train)

best_acc = - np.inf
best_weights = None

train_loss_hist = []
train_acc_hist = []

test_loss_hist = []
test_acc_hist = []

In [ ]:
for epoch in range(n_epochs):
    
    epoch_loss = []
    epoch_acc =[]
    
    model.train()
    
    with tqdm.trange(batches_per_epoch, unit='batch', mininterval=0) as bar:
        bar.set_description(f"Epoch {epoch}")
        
        for i in bar:
            start = i * batch_size
            x_batch = x_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            
            acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))
            
            bar.set_postfix(
            loss=float(loss), 
            acc = float(acc))
            
    model.eval()
    y_pred = model(x_test)
    ce = loss_fn(y_pred, y_test.squeeze())
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_test.squeeze(), 1)).float().mean()
    
    ce = float(ce)
    acc = float(acc)
    
    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    
    test_loss_hist.apped(ce)
    test_acc_hist.append(acc)
    
    if acc > best_acc:
        best_acc = acc
        best_weights = copy.deepcopy(model.state_dict())
        
    print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")